In [ ]:
!pip install rank_eval

In [ ]:
!pip install ranx

In [ ]:
from rank_eval import Qrels, Run, evaluate, compare
import pandas as pd
from pandas import DataFrame


In [ ]:
df = pd.read_csv('../data/resp_classfied_NGC10W_cosseno.csv')
#df = pd.read_csv('resp_elasticsearch_bm25_2.csv')
df['indice'] = df['indice'].astype(str)
df['sugerido_1'] = df['sugerido_1'].astype(str)
df['sugerido_2'] = df['sugerido_2'].astype(str)
df['sugerido_3'] = df['sugerido_3'].astype(str)
df['sugerido_4'] = df['sugerido_4'].astype(str)
df['sugerido_5'] = df['sugerido_5'].astype(str)
df['sugerido_6'] = df['sugerido_6'].astype(str)
df['num_tema_cadastrado'] = df['num_tema_cadastrado'].astype(str)
df['relevancia_tema_cadastrado']=10

In [ ]:
print(len(df))

In [ ]:
print(df.head())

In [ ]:
q_id = []
doc_id = []
score = []
for indice, linha in df.iterrows():
    for i in range(1,7):
        q_id.append(linha[0])
        sug = f"sugerido_{i}"
        doc_id.append(int(linha[sug]))
        sim = f"similaridade_{i}"
        score.append(linha[sim])
    


In [ ]:
print(len(score))

In [ ]:
#len(q_id)
#len(doc_id)
len(score)

In [ ]:
ranx_dict ={"q_id": q_id,
            "doc_id": doc_id,
            "score" : score
}


In [ ]:
qrel = Qrels.from_df(df, q_id_col='indice', doc_id_col='num_tema_cadastrado', score_col='relevancia_tema_cadastrado')

In [ ]:
print(len(qrel))

In [ ]:
run_df = DataFrame.from_dict(ranx_dict)
run_df['q_id'] = run_df['q_id'].astype(str)
run_df['doc_id'] = run_df['doc_id'].astype(str)
run = Run.from_df(
    df=run_df,
    q_id_col="q_id",
    doc_id_col="doc_id",
    score_col="score"
)
run.name = "my_run"

In [ ]:
def recallAtK(k,df):
    Relevant_retrieved = 0
    Total_relevant = len(df)
    for indice,linha in df.iterrows():
        if(0 < linha['posicao_tema_real'] <= k):
            Relevant_retrieved +=1
    return Relevant_retrieved/Total_relevant
        
    

In [ ]:
recall = recallAtK(6,df)
print("recall@6")
print(round(recall,5))

In [ ]:
#print(list(Qrels.keys(qrel)))

In [ ]:
#print(list(Run.keys(run)))

In [ ]:
dicionario = evaluate(qrel, run, ["map@6","ndcg@6","mrr"])

In [ ]:
# Computed metric scores are saved in the Run object
run.mean_scores




In [ ]:
# Access scores for each query
#dict(run.scores)


In [ ]:
f1_score = (2*dicionario['map@6']*recall)/(dicionario['map@6']+recall)

In [ ]:
# Compare different runs and perform statistical tests
#report = compare(
#    qrels=qrels,
#    runs=[run_1, run_2, run_3, run_4, run_5],
#    metrics=["map@100", "mrr@100", "ndcg@10"],
#    max_p=0.01  # P-value threshold
#)

#print(report)

In [ ]:
print(round(f1_score,5))